# Livrable 2 - Groupe 1

## Contenu du livrable

Le but est de traiter un ensemble de photographies afin de les rendre mieux traitables par les algorithmes de Machine Learning. Le traitement à réaliser est une opération de débruitage. Ces algorithmes s'appuieront sur les auto-encodeurs à convolution, et les appliqueront pour améliorer la qualité de l'image.

1. Chargement des données provenant de l'EDA (livrable 1)
2. Création du dataset
3. Définition de l'autoencodeur (CAE)
4. Entrainement
5. Métriques


## Chargement des bibliothèques

In [ ]:
import os
import gdown
import zipfile
import datetime

import PIL
import imghdr
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Sélection de la source de données


In [ ]:
# ID du fichier (extrait de l'URL)
file_id = "1PGTFqsXHRCXV3R2Rns6yz0QS26ihfgYM"
dataset_path = "dataset_livrable_2"
zip_path = dataset_path + ".zip"
extract_dir = pathlib.Path(zip_path).parent / dataset_path
reduce_dataset = True

if not os.path.exists(extract_dir):
    print(f"Le dossier '{extract_dir}' n'existe pas. Téléchargement en cours...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_path, quiet=False)

    print(f"Extraction ZIP en cours...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extraction Zip terminée")
else:
    print(f"Le dossier '{extract_dir}' existe déjà. Téléchargement et extraction non nécessaires.")

data_dir = extract_dir
print(f"Dataset disponible dans : {data_dir}")

## Création du dataset

In [ ]:
validation_split = 0.2
seed = 42

batch_size = 128
img_height = 256
img_width = 256

In [ ]:
# Load the dataset using image_dataset_from_directory

train_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    labels=None,
    shuffle=False
)

val_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    labels=None,
    shuffle=False
)

### Image dimensions

In [ ]:
for images in val_set.take(1):
    print(f"Image dimensions: {images.shape}")
    break

## Visualisation

In [ ]:
def displayImagesFromDataset(dataset, label):
    plt.figure(figsize=(15, 15))
    for images in dataset.take(1):
        for i in range(5):
            ax = plt.subplot(1, 5, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(label)
            plt.axis("off")

In [ ]:
displayImagesFromDataset(train_set, "Train Image")

## Création des différents bruits 

In [ ]:
def add_gaussian_noise(image, mean=0.0, stddev=20.0):
    noise = tf.random.normal(shape=tf.shape(image), mean=mean, stddev=stddev, dtype=tf.float32)
    image = tf.cast(image, tf.float32)
    noisy_image = image + noise
    noisy_image = tf.clip_by_value(noisy_image, 0.0, 255.0)
    noisy_image = tf.cast(noisy_image, tf.uint8)
    return noisy_image

def add_poisson_noise(image):
    image = tf.cast(image, tf.float32)
    # Ajout d'une petite constante pour éviter poisson(0)
    noisy = tf.random.poisson([], lam=image + 1e-3, dtype=tf.float32)
    noisy = tf.clip_by_value(noisy, 0.0, 255.0)
    return tf.cast(noisy, tf.uint8)

def add_salt_pepper_noise(image, prob=0.02):
    image = tf.cast(image, tf.float32)
    rnd = tf.random.uniform(tf.shape(image), 0, 1)
    
    salt = tf.cast(rnd < prob / 2, tf.float32) * 255
    pepper = tf.cast(rnd > 1 - prob / 2, tf.float32) * 0
    mask = tf.cast((rnd >= prob / 2) & (rnd <= 1 - prob / 2), tf.float32)

    noisy_image = image * mask + salt + pepper
    noisy_image = tf.clip_by_value(noisy_image, 0.0, 255.0)
    return tf.cast(noisy_image, tf.uint8)

def add_uniform_noise(image, minval=-20.0, maxval=20.0):
    image = tf.cast(image, tf.float32)
    noise = tf.random.uniform(tf.shape(image), minval=minval, maxval=maxval, dtype=tf.float32)
    noisy_image = image + noise
    noisy_image = tf.clip_by_value(noisy_image, 0.0, 255.0)
    return tf.cast(noisy_image, tf.uint8)

### Affichage des différents bruits

In [ ]:
# Display the same image with different types of noise
def display_noisy_images(image, noise_functions):
    plt.figure(figsize=(15, 10))
    plt.subplot(1, len(noise_functions) + 1, 1)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title("Original")
    plt.axis("off")

    for i, noise_func in enumerate(noise_functions):
        noisy_image = noise_func(image)
        plt.subplot(1, len(noise_functions) + 1, i + 2)
        plt.imshow(noisy_image.numpy().astype("uint8"))
        plt.title(noise_func.__name__)
        plt.axis("off")

    plt.show()
# Display a sample image with different types of noise
sample_image = next(iter(train_set.take(1)))[0]  # Get the first image from the first batch
noise_functions = [add_gaussian_noise, add_poisson_noise, add_salt_pepper_noise, add_uniform_noise]
display_noisy_images(sample_image, noise_functions)

In [ ]:
def random_noise(image):
    noise_type = tf.random.uniform([], 0, 4, dtype=tf.int32)
    
    def case0(): return add_gaussian_noise(image)
    def case1(): return add_salt_pepper_noise(image)
    def case2(): return add_uniform_noise(image)
    def case3(): return add_poisson_noise(image)

    return tf.switch_case(noise_type, branch_fns=[case0, case1, case2, case3])

## Préparation

In [ ]:
noisy_train_set = train_set.map(lambda x: random_noise(x))
noisy_val_set = val_set.map(lambda x: random_noise(x))

In [ ]:
plt.figure(figsize=(15, 15))
for images in train_set.take(1):
    for i in range(5):
        ax = plt.subplot(1, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title("Train Image")
        plt.axis("off")
plt.figure(figsize=(15, 15))
for images in noisy_train_set.take(1):
    for i in range(5):
        ax = plt.subplot(1, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title("Noisy Image")
        plt.axis("off")

## Performance & pre processing

In [ ]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE

# a_train_set = noisy_train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# a_val_set = val_set.cache().prefetch(buffer_size=AUTOTUNE)

for image in noisy_train_set.take(1):
    print(f"Image shape: {image.shape}")

for image in noisy_val_set.take(1):
    print(f"Image shape: {image.shape}")

## Modélisation

In [ ]:
# Configurations principales de nos modèles
IMG_SIZE          = img_width
NB_EPOCHS_DENOISE = 100               # nombre epoch alogithme debruiter
BATCH_SIZE        = 128               # taille batch de traitement
SAV_MODEL_DENOISE = "denoiser.h5"     # sauvegarde du modele de debruitage
LATENT_DIM        = 32

## Encodeur

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Sequential
from keras.regularizers import l1_l2

# Create a Sequential model
encoder = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    # layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    MaxPooling2D((2, 2)),
    # layers.Dropout(0.2),  # Dropout after the first pooling layer
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    MaxPooling2D((2, 2), padding='same'),
    # layers.Dropout(0.3),  # Dropout after the second pooling layer
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    MaxPooling2D((2, 2))
])

## Décodeur

In [ ]:
from keras.models import Sequential

# Decoding
decoder = Sequential([
    Input(shape=encoder.output_shape[1:]),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    UpSampling2D((2, 2)),
    Conv2D(3, (1, 1), activation='sigmoid', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
])


In [ ]:
from tensorflow.keras.models import Model

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = encoder
    self.decoder = decoder

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(LATENT_DIM)

In [ ]:
autoencoder.compile(optimizer='adam',
                    loss='binary_crossentropy')
# autoencoder.summary()

encoder.summary()
decoder.summary()

## Entrainement

In [ ]:
# log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='./L1_model.keras', save_best_only=True)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the model
import tensorflow as tf

paired_train_set = tf.data.Dataset.zip((noisy_train_set, train_set))
paired_val_set = tf.data.Dataset.zip((noisy_val_set, val_set))

paired_train_set = paired_train_set.map(lambda noisy, clean: (tf.cast(noisy, tf.float32) / 255.0, clean / 255.0))
paired_val_set = paired_val_set.map(lambda noisy, clean: (tf.cast(noisy, tf.float32) / 255.0, clean / 255.0))

# Train the autoencoder
history = autoencoder.fit(
    paired_train_set,
    epochs=NB_EPOCHS_DENOISE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=(paired_val_set),
    callbacks=[early_stopping, model_checkpoint] #tensorboard_callback
)

## Métriques

- Courbe d'apprentisssage

In [ ]:
# Visualisation des pertes d'apprentissage (Train) et de validation (Test)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

In [ ]:
# Visualize noisy and denoised images side by side in columns
plt.figure(figsize=(15, 15))
for noisy_images_batch, clean_images_batch in paired_val_set.take(1):
    for i in range(5):  # Display 5 images
        # Display noisy image
        ax = plt.subplot(5, 2, 2 * i + 1)
        noisy_image = noisy_images_batch[i].numpy() * 255.0  # Scale back to [0, 255]
        plt.imshow(noisy_image.astype("uint8"))
        plt.title("Noisy Image")
        plt.axis("off")

        # Display denoised image
        denoised_image = autoencoder(noisy_images_batch[i:i+1])  # Predict denoised image
        denoised_image = denoised_image[0].numpy() * 255.0  # Scale back to [0, 255]
        ax = plt.subplot(5, 2, 2 * i + 2)
        plt.imshow(denoised_image.astype("uint8"))
        plt.title("Denoised Image")
        plt.axis("off")